<a href="https://colab.research.google.com/github/thejsw/Quant/blob/portfolio/PythonPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install numpy-financial

In [24]:
!pip install pandas_datareader

In [33]:
!pip install yfinance

In [35]:
import numpy_financial as npf
import numpy as np
import pandas as pd
import yfinance as yf
import random
import math

# 1. 파이썬과 재무 기초 지식

## 1.2 현금흐름, 이자율과 시간 가치

이자율은 화폐를 보유하는 경우에 생기는 기회비용  
단리: 원금 x (1 + 이율 x 기간)  
복리: 원금 x (1 + 이율) ** 기간

In [ ]:
a = 1
r = 1.0

n = 4
c_compound = a * (1+r / n) ** n

c_compound

## 1.3 NPV와 IRR

**NPV(순현재가치)**: 투자로부터 예상되는 미래 현금 흐름을 현재 가치로 환산한 후, 초기 투자 비용을 차감한 값  
-> NPV가 0보다 크면 투자할 가치가 있다고 판단함

In [ ]:
# 현금흐름을 cashflows에 저장, i는 횟수, r은 이자율
cashflows = [12000, 15000, 18000, 21000, 26000]
i = 0
r = 0.015

# 최초 투자금액, 현금 유출이므로 (-)
npv = -70000

# cashflows 리스트를 반복해 미래가치를 현재가치로 계산해서 npv 변수에 누적
for c in cashflows:
  i = i+1
  npv = npv + c/(1+r) ** i

npv


In [ ]:
# numpy_financial 라이브러리 사용
cashflows = [-70000, 12000, 15000, 18000, 21000, 26000]
r = 0.015

npv = npf.npv(r, cashflows)
npv

**IRR(내부수익률)**: 투자로부터 예상되는 현금 흐름의 현재 가치와 투자 비용이 같아지도록 만드는 할인율  
-> 투자를 통해 얻을 수 있는 수익률  
-> NPV를 0으로 만듦  
-> 예를 들어, IRR은 약 15.24%일 경우, 투자자가 요구하는 최소 수익률이 10%라면, 이 프로젝트는 투자할 가치가 있다고 판단

In [ ]:
cashflows = [-70000, 12000, 15000, 18000, 21000, 26000]

irr = npf.irr(cashflows)
npv = npf.npv(irr, cashflows)

print('IRR {0:.1%} makes NPV {1:.0f}'.format(irr, npv))

## 1.4 수익률 대 수익률

**산술평균 (Arithmetic Mean)**  
일반적으로 "평균"이라고 부르는 값으로, 모든 값을 더한 후 값의 개수로 나눈 값  
  
-> 계산이 간단하고 직관적이나,   
복리 효과를 고려하지 않기 때문에 장기 투자 성과를 정확하게 반영X  
특히 수익률 변동성이 큰 경우, 실제 투자 수익률과 차이가 클 수 있음

**기하평균 (Geometric Mean)**  
값들을 모두 곱한 후 값의 개수에 해당하는 제곱근을 취한 값  
-> 투자 성과를 정확하게 측정하고, 여러 투자 안의 성과를 비교하는 데 유용

**지배원리 (dominance principle)**  
포트폴리오에서 얻는 기대수익률은 클수록 좋고, 표준편차와 분산으로 표현하는 위험은 작을수록 좋음

## 1.5 자주 사용하는 통계량

**이동평균**  
계산에 들어가는 값 중 가장 오래된 값을 버리고, 새로운 값을 추가하여 구한 평균

In [ ]:
prices = [44800, 44850, 44600, 43750, 44000, 43900, 44350, 45350, 45500, 45700]

# 5일 이동평균
n = 5

for p in prices[n: ]:
  end_index = prices.index(p)
  begin_index = end_index - n
  print(sum(prices[begin_index:end_index])/n)

**가중평균**  
자료에 가중치를 고려해 구한 평균  
-> 예를 들어, A종목 100주를 주당 5,000원에 매입하고, 다시 50주를 7,500원에 추가 매수했다면,  
5,000원 매입단가의 비중과 7,500원 매입단가의 비중을 반영해 가중평균으로 계산

In [ ]:
# sizes는 매입수량, prices는 매입단가
sizes = [100, 50, 80]
prices = [5,000, 7,500, 6,000]

# wgt_avg는 합계를 저장할 변수
wgt_avg = 0.0

# sizes와 prices를 zip함수로 묶어 for 루프로 반복함
for s, p in zip(sizes, prices):
  wgt_avg += s*p

print(wgt_avg)

**공분산**   
두 변수 간의 선형적인 관계  
Cov(X, Y) = Σ [(Xi - X̄) * (Yi - Ȳ)] / (n - 1)  

**상관관계**  
공분산의 한계를 극복하기 위해 상관관계라는 개념을 사용  
공분산을 각 변수의 표준편차로 나누어 -1과 1 사이의 값으로 정규화

In [ ]:
# 평균을 계산하는 함수
def mean(x):
  return sum(x) / len(x)

# 두 리스트 곱의 합계
def sum_of_product(xs, ys):
  return sum(x*y for x, y in zip(xs, ys))

# 제곱합을 계산하는 함수
def sum_of_squares(y):
  return sum_of_squares(v, v)

# 편차를 계산하는 함수
def deviation(xs):
  x_mean = mean(xs)
  return [x - x_mean for x in xs]

# 분산을 계산하는 함수
def variance(x):
  n = len(x)
  deviations = deviation(x)
  return sum_of_squares(deviations) / (n-1)

# 공분산을 계산하는 함수
def convariance(x, y):
  n = len(x)
  deviations = deviation(x)
  return sum_of_squares(deviations) / (n-1)

# 표준편차를 계산하는 함수
def standard_deviation(x):
  return math.sqrt(variance(x))

# 상관계수를 계산하는 함수
def correlation(xs, ys):
  stdev_x = standard_deviation(xs)
  stdev_y = standard_deviation(ys)
  if stdev_x > 0 and stdev_y > 0:
    return convariance(xs, ys) / (stdev_x, stdev_y)
  else:
    return 0

# 2. 투자와 자산배분

> 자산은 리스크가 없는 자산과 리스크가 있는 자산으로 나뉜다.   
> 리스크가 없는 자산으로만 구성된 포트폴리오는 투자자가 원하는 기대수익률을 만족시켜주지 못한다.  
> 따라서 자산을 위험자산과 무위험자산으로 분류하고 *좋은 위험자산 포트폴리오를 찾아내는*, **효율적 포트폴리오**를 구성해야 한다.



## 2.3 포트폴리오 성과 측정 삼총사

**1. 샤프지수**  
- 개념: 샤프 지수는 총 위험(변동성) 대비 초과 수익률을 나타내는 지표  
즉, 투자자가 감수한 위험 대비 얼마나 많은 초과 수익을 얻었는지 측정  

`샤프 지수 = (포트폴리오 수익률 - 무위험 수익률) / 포트폴리오 표준편차(총 위험)`  

- 해석: 높은 샤프 지수는 위험 대비 수익률이 높다는 것을 의미함. 일반적으로 샤프 지수가 1 이상이면 좋은 투자로 간주  

**2. 젠센알파지수**  
- 개념: 포트폴리오의 실제 수익률과 기대수익률의 차이  
즉, 시장 대비 얼마나 높은 성과를 냈는지 측정

`젠센 알파 = 포트폴리오 수익률 - 기대(적정)수익률`  

- 해석: 양수이면 포트폴리오가 CAPM(자본자산가격결정모델)에서 예측하는 것보다 높은 수익률을 달성

**3. 트레이너지수**  
- 개념: 체계적 위험(베타) 대비 초과 수익률을 나타내는 지표
즉, 투자자가 감수한 체계적 위험 단위당 얼마나 많은 초과 수익을 얻었는지 측정

`트레이너 지수 = (포트폴리오 수익률 - 무위험 수익률) / β`  
베타: 시장 수익률 변동에 대한 포트폴리오 수익률의 민감도(체계적 위험)  

- 해석: 높은 트레이너 지수는 체계적 위험 대비 수익률이 높다는 것을 의미

**분산 투자가 잘 되어 있지 않은 포트폴리오**: 샤프 지수가 더 적합  
**분산 투자가 잘 되어 있고, 체계적 위험이 중요한 포트폴리오**: 트레이너 지수가 더 적합  
**특정 벤치마크와 비교하여 초과 수익을 평가하고 싶을 때**: 젠센 알파 지수가 더 적합  

**정보비율**  
투자 성과를 평가하는 지표  
정보비율이 높을수록 더 많은 초과 수익을 얻음

`정보비율 (IR) = 초과 수익률(포트폴리오 수익률 - 벤치마크 수익률) / 추적 오차(표준편차)`





**최대 낙폭(MDD)**  
고점에서 저점까지 하락폭이 가장 큰 구간의 등락률  
즉, 투자자의 최대손실가능 수익률을 의미

`MDD = 기간 중 최저 가치 - 최고 가치 / 최고 가치`

In [ ]:
values = [100, 120, 130, 100, 65, 80, 100, 120, 140, 160]

def mdd(x):
  arr = np.array(x)
  idx_lower = np.argmin(arr - np.maximum.accumulate(arr))
  idx_upper = np.argmax(arr[:idx_lower])
  return (arr[idx_lower] - arr[idx_upper]) / arr[idx_upper]

print("{:.2%}".format(mdd(values)))

# 3. 평균-분산 포트폴리오 이론

## 3.1 포트폴리오의 기대수익과 위험

> 노벨 경제학상 수상자 해리 마코위츠는 투자자들이 주가 대폭락 이후 수익뿐만 아니라 리스크에도 관심을 가지는 것을 목격했다.  
그는 **상관계수가 낮은 자산을 결합**해 위험을 줄이며 수익률을 높일 수 있는 최적 포트폴리오를 제시했다.


### 3.1.1 두 개 주식으로 구성된 포트폴리오
확률분포 없이 조건을 단순화해 포트폴리오의 기대수익률을 구해볼 수 있다

**1. 포트폴리오 기대수익률** = A 투자 비중 * A 기대수익률 + B 투자 비중 * B 기대수익률

In [ ]:
# 각각 호황시, 평상시, 불황시 수익률과 확률
stock_a = [0.07, 0.04, 0.02]
stock_b = [0.15, 0.07, -0.07]
prob = [1/3, 1/3, 1/3]

# 주식 a,b의 경기 국면에 따른 수익률 기댓값을 저장할 변수
ex_a = 0.0
ex_b = 0.0
wgt_a = 0.5
wgt_b = 0.5

# 주식 a,b의 기대수익률
ex_a = sum(s*p for s,p in zip(stock_a, prob))
ex_b = sum(s*p for s,p in zip(stock_b, prob))

# 포트폴리오 기대수익률
ex_portfolio = wgt_a * ex_a + wgt_b * ex_b

print('주식 a의 기대수익률은 {:.2%}'.format(ex_a))
print('주식 b의 기대수익률은 {:.2%}'.format(ex_b))
print('포트폴리오 기대수익률은 {:.2%}'.format(ex_portfolio))

**2. 행렬 연산**으로 구해보는 포트폴리오 기대수익률

In [ ]:
# matrix()를 통해 확률, 주식 a,b의 수익률을 1x3 행렬로 만듦
stock_a = np.array([[0.07, 0.04, 0.02]])
stock_b = np.array([[0.15, 0.07, -0.07]])
prob = np.array([[1/3, 1/3, 1/3]])

# 행렬 곱하기 연산 수행, 행렬 차원을 맞추기 위해 전치 사용
# prob는 1x3, stock_a, stock_b는 3x1이 되어 1x1의 행렬이 나오게 됨
ex_a = prob @ stock_a.T
ex_b = prob @ stock_b.T
ex_portfolio = wgt_a * ex_a + wgt_b * ex_b

print('주식 a의 기대수익률은 {:.2%}'.format(ex_a[0, 0]))
print('주식 b의 기대수익률은 {:.2%}'.format(ex_b[0, 0]))
print('포트폴리오 기대수익률은 {:.2%}'.format(ex_portfolio[0, 0]))

**3. 포트폴리오 위험** = 주식A 투자 비중** x 주식A 분산 + 주식B 투자비중** + 주식B 분산 + 2 x 주식A 투자 비중 x 주식B 투자 비중 x 포트폴리오 공분산

In [ ]:
# 1. 변수 선언 및 주식 a,b의 분산 구하기

# 각각 호황시, 평상시, 불황시 수익률과 확률
stock_a = [0.07, 0.04, 0.02]
stock_b = [0.15, 0.07, -0.07]
prob = [1/3, 1/3, 1/3]

# 주식 a,b의 경기 국면에 따른 수익률 기댓값을 저장할 변수
ex_a = 0.0
ex_b = 0.0
wgt_a = 0.5
wgt_b = 0.5

# 분산을 저장할 변수 준비
var_a, var_b = 0.0, 0.0

# 확률과 편차 제곱을 곱해 각각 분산으로 저장
var_a = sum(p*(s-ex_a)**2 for p,s in zip(prob, stock_a))
var_b = sum(p*(s-ex_b)**2 for p,s in zip(prob, stock_b))

print('주식 a의 분산은 {:.2%}'.format(var_a))
print('주식 b의 분산은 {:.2%}'.format(var_b))

In [ ]:
# 2. 포트폴리오 위험(분산) 측정을 위한 공분산, 표준편차 계산

cov = sum(p*(a-ex_a)*(b-ex_b) for p,a,b in zip(prob, stock_a, stock_b))
var_p = wgt_a**2 * var_a + wgt_b**2 * var_b + 2*wgt_a*wgt_b*cov
std_p = math.sqrt(var_p)

print('포트폴리오의 위험(분산)은 {:.2%}'.format(var_p))
print('포트폴리오의 표준편차는 {:.2%}'.format(std_p))

### 3.1.2 Pandas를 이용해 실제 데이터로 포트폴리오 기대수익률 계산

In [40]:
tickers = ['MMM', 'ADBE', 'AMD', 'GOOGL', 'GOOG', 'AMZN']

adjClose = pd.DataFrame()

adjClose = yf.download(tickers, start='2023-01-01')

dailySimpleReturns = adjClose.pct_change()
meanReturns = np.matrix(dailySimpleReturns.mean()).T

print(meanReturns)

[*********************100%***********************]  6 of 6 completed


[[0.0003234 ]
 [0.00110254]
 [0.0014983 ]
 [0.00117094]
 [0.00115688]
 [0.00085717]
 [0.00028714]
 [0.00099477]
 [0.00149566]
 [0.00112892]
 [0.00111441]
 [0.00089654]
 [0.00031585]
 [0.00100364]
 [0.00150924]
 [0.00114952]
 [0.00113291]
 [0.00082772]
 [0.00034095]
 [0.00106239]
 [0.00156303]
 [0.00117371]
 [0.00115597]
 [0.0008821 ]
 [0.06545773]
 [0.04713031]
 [0.04763657]
 [0.05585554]
 [0.05556571]
 [0.11177763]]


## 3.3 체계적 위험과 비체계적 위험  

- 비체계적 위험: 분산 투자로 제거할 수 있는 위험  
개별 주식과 관련된 고유의 위험으로, 어닝쇼크(실적부진), 소송, 노사분규와 같은 것
- 체계적 위험: 분산 투자로 제거할 수 없는 위험  
시장 전체와 관련된 위험으로 경제변수(이자율, 환율, 경기선행지수, 실업률, 경제정책 등)의 불리한 움직임

## 3.4 무위험자산과 최적 자산배분  

- 무위험자산: 이자율, 인플레이션의 변화에도 영향을 받지 않아, 미래의 현금흐름에 불확실성이 없는 자산을 의미함  
예를 들어, 만기가 짧아 인플레이션, 이자율의 영향을 적게 받는 정부발행채권,  
또는 인플레이션의 영향을 받지만 만기가 긴 10년 국공채가 있음